### Imports

In [1]:
import pandas as pd
import numpy as np
import pymongo
import json
import collections
import math
from datetime import date, datetime
import cx_Oracle

# path da instalação do instantclient
cx_Oracle.init_oracle_client(lib_dir=r"C:\instantclient_21_9")

### Estabelecer conexão

In [2]:
# Conectar ao Oracle
dsn_tns = cx_Oracle.makedsn('localhost', '1521', service_name='xe')
conn = cx_Oracle.connect(user='store', password='uminho2023', dsn=dsn_tns)

### Selecionar dados das tabelas

In [3]:
oracle_cursor = conn.cursor()

----
Grupo 1: USER

STORE_USERS

In [4]:
oracle_cursor.execute("SELECT * FROM STORE_USERS")
results_storeUsers = oracle_cursor.fetchall()

store_users = pd.DataFrame(results_storeUsers, columns=[i[0] for i in oracle_cursor.description])

ADDRESSES

In [5]:
oracle_cursor.execute("SELECT * FROM ADDRESSES")
results_addresses = oracle_cursor.fetchall()

addresses = pd.DataFrame(results_addresses, columns=[i[0] for i in oracle_cursor.description])

----
Grupo 2: EMPLOYEES

EMPLOYEES

In [6]:
oracle_cursor.execute("SELECT * FROM EMPLOYEES")
results_employees = oracle_cursor.fetchall()

employees = pd.DataFrame(results_employees, columns=[i[0] for i in oracle_cursor.description])

DEPARTMENTS

In [7]:
oracle_cursor.execute("SELECT * FROM DEPARTMENTS")
results_departments = oracle_cursor.fetchall()

departments = pd.DataFrame(results_departments, columns=[i[0] for i in oracle_cursor.description])

EMPLOYEES_ARCHIVE

In [8]:
oracle_cursor.execute("SELECT * FROM EMPLOYEES_ARCHIVE")
results_employeesArchive = oracle_cursor.fetchall()

employeesArchive = pd.DataFrame(results_employeesArchive, columns=[i[0] for i in oracle_cursor.description])

----
Grupo 3: PRODUCT  

PRODUCT

In [9]:
oracle_cursor.execute("SELECT * FROM PRODUCT")
results_product = oracle_cursor.fetchall()

product = pd.DataFrame(results_product, columns=[i[0] for i in oracle_cursor.description])

PRODUCT_CATEGORIES

In [10]:
oracle_cursor.execute("SELECT * FROM PRODUCT_CATEGORIES")
results_productCategories = oracle_cursor.fetchall()

productCategories = pd.DataFrame(results_productCategories, columns=[i[0] for i in oracle_cursor.description])

DISCOUNT

In [11]:
oracle_cursor.execute("SELECT * FROM DISCOUNT")
results_discount = oracle_cursor.fetchall()

discount = pd.DataFrame(results_discount, columns=[i[0] for i in oracle_cursor.description])

STOCK

In [12]:
oracle_cursor.execute("SELECT * FROM STOCK")
results_stock = oracle_cursor.fetchall()

stock = pd.DataFrame(results_stock, columns=[i[0] for i in oracle_cursor.description])

----
Grupo 4: ORDER  
  
  
PAYMENT_DETAILS

In [13]:
oracle_cursor.execute("SELECT * FROM PAYMENT_DETAILS")
results_paymentDetails = oracle_cursor.fetchall()

payment_details = pd.DataFrame(results_paymentDetails, columns=[i[0] for i in oracle_cursor.description])

ORDER_DETAILS

In [14]:
oracle_cursor.execute("SELECT * FROM ORDER_DETAILS")
results_orderDetails = oracle_cursor.fetchall()

order_details = pd.DataFrame(results_orderDetails, columns=[i[0] for i in oracle_cursor.description])

ORDER_ITEMS

In [15]:
oracle_cursor.execute("SELECT * FROM ORDER_ITEMS")
results_orderItems = oracle_cursor.fetchall()

order_items = pd.DataFrame(results_orderItems, columns=[i[0] for i in oracle_cursor.description])

CART_ITEM

In [16]:
oracle_cursor.execute("SELECT * FROM CART_ITEM")
results_cartItem = oracle_cursor.fetchall()

cart_item = pd.DataFrame(results_cartItem, columns=[i[0] for i in oracle_cursor.description])

SHOPPING_SESSION

In [17]:
oracle_cursor.execute("SELECT * FROM SHOPPING_SESSION")
results_shoppingSession = oracle_cursor.fetchall()

shopping_session = pd.DataFrame(results_shoppingSession, columns=[i[0] for i in oracle_cursor.description])

----
# Tratar os dados

----
### 1. USER
- Store\_users em que cada user contem informação dos seus Addresses
- Adicionar o campo shopping_session: Contém as sessões criadas pelo utilizador.
- Adicionar o campo cart_item dentro da shopping_session: Contém os produtos adicionados ao carrinho pelo cliente numa determinada
sessão.

### Iniciar conexão com o MongoDB

In [18]:
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Create a database called "oes"
database = client["oes"]

# Create collections
collection_user = database["user"]
collection_product = database["product"]
collection_order = database["order"]
collection_employees = database["employees"]

USER - Enviar para o Mongo

In [19]:
user_documents = []

for index, row in store_users.iterrows():
    userObj = {
        "user_ID": row['USER_ID'],
        "user_firstName": row['FIRST_NAME'],
        "user_middleName": row['MIDDLE_NAME'] if row['MIDDLE_NAME'] else '',
        "user_lastName": row['LAST_NAME'],
        "user_phoneNumber": row['PHONE_NUMBER'],
        "user_email": row['EMAIL'],
        "user_username": row['USERNAME'],
        "user_password": row['USER_PASSWORD'],
        "user_registeredAt": row['REGISTERED_AT'].strftime('%Y-%m-%d %H:%M:%S') if not pd.isnull(row['REGISTERED_AT']) else '2022-07-01 09:57:48',
    }
    
    user_documents.append(userObj)
    
collection_user.insert_many(user_documents)    

- Adicionar o campo Shopping_session
- Adicionar o campo cart_item às shopping_sessions do user

In [20]:
# Dicionário para armazenar os arrays de sessões de para cada user
sessions_por_user = collections.defaultdict(list)

for index, row in shopping_session.iterrows():
    userID = row['USER_ID']
    
    newShoppingSession = {
        "session_ID": row['SESSION_ID'],
        "session_userID": row['USER_ID'],
        "session_createdAt": row['CREATED_AT'].strftime('%Y-%m-%d %H:%M:%S') if not pd.isnull(row['CREATED_AT']) else '2022-07-01 09:57:48',
        "session_modifiedAt": row['MODIFIED_AT'].strftime('%Y-%m-%d %H:%M:%S') if not pd.isnull(row['MODIFIED_AT']) else '2022-07-01 09:57:48'
    }

    # Adicionar a sessão ao array de sessões associadas ao user
    sessions_por_user[userID].append(newShoppingSession)


for index, row in cart_item.iterrows():
    sessionID = row['SESSION_ID']
    session_row = shopping_session.loc[shopping_session["SESSION_ID"] == sessionID].iloc[0]
    userID = session_row['USER_ID']
    

    newCartItem = {
        "cartItem_ID": row['CART_ITEM_ID'],
        "cartItem_sessionID": row['SESSION_ID'],
        "cartItem_productID": row['PRODUCT_ID'],
        "cartItem_quantity": row['QUANTITY'],
        "cartItem_createdAt": row['CREATED_AT'].strftime('%Y-%m-%d %H:%M:%S') if not pd.isnull(row['CREATED_AT']) else '2022-07-01 09:57:48',
        "cartItem_modifiedAt": row['MODIFIED_AT'].strftime('%Y-%m-%d %H:%M:%S') if not pd.isnull(row['MODIFIED_AT']) else '2022-07-01 09:57:48'
    }

    # Find the user's session array and append newCartItem
    for session in sessions_por_user[userID]:
        if session["session_ID"] == sessionID:
            session.setdefault("cart_items", []).append(newCartItem)
            break


# Atualizar cada USER com o array de sessões associadas
for userID, sessions in sessions_por_user.items():
    collection_user.update_one({"user_ID": userID}, {'$set': {"shopping_sessions": sessions}})


----
### 2. EMPLOYEES
- Passar a ter um único documento para employees
- Adicionar um campo de "estado" (antigo/atual)
- Adicionar um campo de "department_info"


In [22]:
# Iterar pelo EMPLOYEES_ARCHIVE e descontruir em 2 registos (um com o estado "old", e o outro com o estado "active")

old_employees = []
active_employees = []

for index, row in employeesArchive.iterrows():
    
    if row['OLD_EMPLOYEE_ID'] is not None and not np.isnan(row['OLD_EMPLOYEE_ID']):
        employee_info_old = {
        "emp_ID": row['OLD_EMPLOYEE_ID'],
        "emp_firstName": row['OLD_FIRST_NAME'],
        "emp_MiddleName": row['OLD_MIDDLE_NAME'] if row['OLD_MIDDLE_NAME'] else '',
        "emp_LastName": row['OLD_LAST_NAME'],
        "emp_dateOfBirth": row['OLD_DATE_OF_BIRTH'].strftime('%Y-%m-%d %H:%M:%S'),
        "emp_departmentID": row['OLD_DEPARTMENT_ID'],
        "emp_hireDate": row['OLD_HIRE_DATE'].strftime('%Y-%m-%d %H:%M:%S'),
        "emp_salary": row['OLD_SALARY'],
        "emp_phoneNumber": row['OLD_PHONE_NUMBER'],
        "emp_Email": row['OLD_EMAIL'],
        "emp_ssnNumber": row['OLD_SSN_NUMBER'],
        "emp_managerID": row['OLD_MANAGER_ID'] if not math.isnan(row['OLD_MANAGER_ID']) else 0,
        "employee_state": "old"
    }
        old_employees.append(employee_info_old)

    employee_info_active = {
            "emp_ID": row['NEW_EMPLOYEE_ID'],
            "emp_firstName": row['NEW_FIRST_NAME'],
            "emp_MiddleName": row['NEW_MIDDLE_NAME'] if row['NEW_MIDDLE_NAME'] else '',
            "emp_LastName": row['NEW_LAST_NAME'],
            "emp_dateOfBirth": row['NEW_DATE_OF_BIRTH'].strftime('%Y-%m-%d %H:%M:%S'),
            "emp_departmentID": row['NEW_DEPARTMENT_ID'],
            "emp_hireDate": row['NEW_HIRE_DATE'].strftime('%Y-%m-%d %H:%M:%S'),
            "emp_salary": row['NEW_SALARY'],
            "emp_phoneNumber": row['NEW_PHONE_NUMBER'],
            "emp_Email": row['NEW_EMAIL'],
            "emp_ssnNumber": row['NEW_SSN_NUMBER'],
            "emp_managerID": row['NEW_MANAGER_ID'] if not math.isnan(row['NEW_MANAGER_ID']) else 0,
            "employee_state": "active"
    }
    active_employees.append(employee_info_active)

####

### EMPLOYEES - Enviar para o Mongo

In [23]:
# Convert the active_employees array into a list of employee documents
employee_documents = []
for employee in active_employees:
    employee_documents.append(employee)

# Insert the employee documents into the employees collection
collection_employees.insert_many(employee_documents)

- Adicionar um campo para o departamento

Nota: manager ID que estamos a ir buscar (proveniente do departamento) é diferente daquele que é fornecido no employee_archive

In [24]:
for entry in active_employees:
    id_employee = entry["emp_ID"]

    # Get the department ID for the current employee
    dept_id = entry['emp_departmentID']

    # Find the department information for the current department ID
    dept_info = departments.query("DEPARTMENT_ID == @dept_id").iloc[0]

    if not dept_info.empty:
        departmentId = dept_info["DEPARTMENT_ID"]
        departmentName = dept_info['DEPARTMENT_NAME']
        managerId = dept_info['MANAGER_ID'] if dept_info['MANAGER_ID'] else 0
        departmentDescription = dept_info['DEPARTMENT_DESC']
    
        # Create a new dictionary with the department information
        new_field = {
            "department_id": int(departmentId),
            "department_name": departmentName,
            "manager_id": int(managerId),
            "department_description": departmentDescription
        }

        # Update the "department_info" field in the current employee dictionary
        entry["department_info"] = new_field
        del entry["emp_departmentID"]
        if entry["_id"]: 
            del entry["_id"]
        
        collection_employees.update_one({"emp_ID": id_employee}, {'$set': {"department_info": new_field}})
    else: 
        pass

----
### 3. PRODUCT
- Passar a ter um único documento para products
- Adicionar um campo de Category
- Adicionar um campo de Discount
- Adicionar um campo de Stock

In [25]:
product_documents = []

for index, row in product.iterrows():
    productObj = {
        "product_ID": row['PRODUCT_ID'],
        "product_name": row['PRODUCT_NAME'],
        "product_categoryID": row['CATEGORY_ID'],
        "product_sku": row['SKU'],
        "product_price": row['PRICE'],
        "product_discountId": int(row['DISCOUNT_ID']) if not math.isnan(row['DISCOUNT_ID']) else 0,
        "product_createdAt": row['CREATED_AT'].strftime('%Y-%m-%d %H:%M:%S') if not pd.isnull(row['CREATED_AT']) else '2022-07-01 09:57:48',
        "product_lastModified": row['LAST_MODIFIED'].strftime('%Y-%m-%d %H:%M:%S') if not pd.isnull(row['LAST_MODIFIED']) else '2022-07-01 09:57:48'
    }
    
    product_documents.append(productObj)
    
collection_product.insert_many(product_documents)    

- Adicionar campos category, discount, stock;  
 PRODUCT - Enviar para o Mongo

In [26]:
for row in product_documents:
    
    productID = row['product_ID']
   
    category_row = productCategories.loc[productCategories["CATEGORY_ID"] == row['product_categoryID']].iloc[0]
    new_category = {
        "category_ID": int(category_row['CATEGORY_ID']),
        "category_name": category_row['CATEGORY_NAME'],
    }

    collection_product.update_one({"product_ID": productID}, {'$set': {"product_category": new_category}})

    stock_row = stock.loc[stock["PRODUCT_ID"] == productID].iloc[0]
    new_stock = {
        "quantity": int(stock_row['QUANTITY']),
        "max_stock_quantity": int(stock_row['MAX_STOCK_QUANTITY']),
        "unit": stock_row['UNIT']
    }
    
    collection_product.update_one({"product_ID": productID}, {'$set': {"product_stock": new_stock}})



    if row["_id"]: 
        del row["_id"]


    discount_row = {}
    if row['product_discountId'] != 0:
        discount_row = discount.loc[discount["DISCOUNT_ID"] == row['product_discountId']].iloc[0]
        new_discount = {
            "discount_id": int(discount_row['DISCOUNT_ID']),
            "discount_name": discount_row['DISCOUNT_NAME'],
            "discount_desc": discount_row['DISCOUNT_DESC'],
            "discount_percent": int(discount_row['DISCOUNT_PERCENT']),
            "discount_is_active_status": discount_row['IS_ACTIVE_STATUS'],
            "discount_createdAt": discount_row['CREATED_AT'] if not pd.isnull(discount_row['CREATED_AT']) else '2022-07-01 09:57:48',
            "discount_modifiedAt": discount_row['MODIFIED_AT'] if not pd.isnull(discount_row['MODIFIED_AT']) else '2022-07-01 09:57:48',
        }
        collection_product.update_one({"product_ID": productID}, {'$set': {"product_discount": new_discount}})

----
### 4. ORDER
- Adicionar o campo order_details: Detalhes do pedido do utilizador.
- Adicionar o campo order_items: Contém os produtos do pedido feito pelo utilizador.
- Adicionar o campo payment_details: Contém os detalhes de pagamento do pedido.
- Adicionar o campo address

- order_details

In [27]:
order_documents = []

for index, row in order_details.iterrows():
    orderObj = {
        "order_detailsID": row['ORDER_DETAILS_ID'],
        "order_userID": row['USER_ID'],
        "order_total": row['TOTAL'],
        "order_paymentID": row['PAYMENT_ID'],
        "order_shippingMethod": row['SHIPPING_METHOD'],
        "order_deliveryAddressID": row['DELIVERY_ADRESS_ID'],
        "order_createdAt": row['CREATED_AT'].strftime('%Y-%m-%d %H:%M:%S') if not pd.isnull(row['CREATED_AT']) else '2022-07-01 09:57:48',
        "order_modifiedAt": row['MODIFIED_AT'].strftime('%Y-%m-%d %H:%M:%S') if not pd.isnull(row['MODIFIED_AT']) else '2022-07-01 09:57:48'
    }

    order_documents.append(orderObj)
    
collection_order.insert_many(order_documents)    

-Adicionar campo order_items

In [28]:
# Dicionário para armazenar os arrays de items(objetos) para cada pedido
items_por_order = collections.defaultdict(list)

for index, row in order_items.iterrows():
    orderDetailsID = row['ORDER_DETAILS_ID']
    
    new_orderItem = {
        "item_ID": row['ORDER_ITEMS_ID'],
        "item_detailsID": row['ORDER_DETAILS_ID'],
        "item_productID": row['PRODUCT_ID'],
        "item_createdAt": row['CREATED_AT'].strftime('%Y-%m-%d %H:%M:%S') if not pd.isnull(row['CREATED_AT']) else '2022-07-01 09:57:48',
        "item_modifiedAt": row['MODIFIED_AT'].strftime('%Y-%m-%d %H:%M:%S') if not pd.isnull(row['MODIFIED_AT']) else '2022-07-01 09:57:48'
    }
    
    # Adicionar o item ao array de itens associados àquele pedido
    items_por_order[orderDetailsID].append(new_orderItem)
  
# Atualizar cada ORDER com o array de items associados
for orderDetailsID, items in items_por_order.items():
    collection_order.update_one({"order_detailsID": orderDetailsID}, {'$set': {"order_items": items}})

-Adicionar campo payment_details

In [29]:
for index, row in payment_details.iterrows():
    paymentID = row['PAYMENT_DETAILS_ID']
    
    new_paymentDetails = {
        "payment_detailsID": row['PAYMENT_DETAILS_ID'],
        "payment_orderID": row['ORDER_ID'],
        "payment_amount": row['AMOUNT'],
        "payment_provider": row['PROVIDER'],
        "payment_status": row['PAYMENT_STATUS'],
        "payment_createdAt": row['CREATED_AT'].strftime('%Y-%m-%d %H:%M:%S') if not pd.isnull(row['CREATED_AT']) else '2022-07-01 09:57:48',
        "payment_modifiedAt": row['MODIFIED_AT'].strftime('%Y-%m-%d %H:%M:%S') if not pd.isnull(row['MODIFIED_AT']) else '2022-07-01 09:57:48'
    }

    
    collection_order.update_one({"order_paymentID": paymentID}, {'$set': {"payment_details": new_paymentDetails}})

- Adicionar o campo address

In [30]:
for index, row in addresses.iterrows():
    deliveryAddressID = row['ADRESS_ID']
    
    new_Address = {
        "address_id": row['ADRESS_ID'],
        "line_1": row['LINE_1'],
        "line_2": row['LINE_2'],
        "city": row['CITY'],
        "zip_code": row['ZIP_CODE'],
        "province": row['PROVINCE'],
        "country": row['COUNTRY'],
    }

    
    collection_order.update_one({"order_deliveryAddressID": deliveryAddressID}, {'$set': {"address": new_Address}})